In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import sklearn.metrics as metrics


In [2]:
transform= transforms.Compose([transforms.ToTensor()])

trainset=torchvision.datasets.MNIST(root='./data',train=True,download=True,transform=transform)
trainloader=torch.utils.data.DataLoader(trainset,batch_size=32,shuffle=True,num_workers=2)

testset=torchvision.datasets.MNIST(root='./data',train=False,download=True,transform=transform)
testloader=torch.utils.data.DataLoader(trainset,batch_size=32,shuffle=False,num_workers=2)


In [10]:
trainloader.dataset.shape




AttributeError: 'MNIST' object has no attribute 'shape'

In [4]:
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel,self).__init__()
        
        #input channel 1 : gray images
        #kernel_size : filter sizes
        #
        self.conv1=nn.Conv2d(in_channels=1,out_channels=32,kernel_size=3)
        
        # Linear layer : (input,output)
        self.d1=nn.Linear(26*26*32,128)
        # 10 is the number of classes
        self.d2=nn.Linear(128,10)
        
    def forward(self,x):
        #32x1x248x28 : 32 is batch size
        x=self.conv1(x)
        x=F.relu(x)
        
        #flatten 32 x (32x26x26)
        x=x.flatten(start_dim=1)
        
        # 32x26x26) ==> 322x128
        x=self.d1(x)
        x=F.relu(x)
        
        #32x10
        logits=self.d2(x)
        out=F.softmax(logits,dim=1)
        return out
        
    

In [5]:
learning_rate = 0.001
num_epochs = 5

device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model= MyModel()
model=model.to(device)
criterion= nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)


In [6]:
for epoch in range(num_epochs):
    train_running_loss=0.0
    train_acc=0.0
    
    for i, (images,labels)  in enumerate(trainloader):
        images=images.to(device)
        labels=labels.to(device)
        
        logits = model(images)
        loss=criterion(logits,labels)
        
        #if we don't write this: pytorch accumulate gradiants
        optimizer.zero_grad()
        loss.backward() # computes gradiants
        optimizer.step()
        
        train_running_loss += loss.detach().item()
        train_acc += (torch.argmax(logits,1).flatten()==labels).type(torch.float).mean().item()
    
    print('Epoch: %d , Loss: %.4f , Train ACC: %.2f'\
          %(epoch,train_running_loss/i,train_acc/i))
    
        

Epoch: 0 , Loss: 1.6209 , Train ACC: 0.84
Epoch: 1 , Loss: 1.4925 , Train ACC: 0.97
Epoch: 2 , Loss: 1.4825 , Train ACC: 0.98
Epoch: 3 , Loss: 1.4782 , Train ACC: 0.99
Epoch: 4 , Loss: 1.4750 , Train ACC: 0.99


In [9]:
test_acc= 0.00
for i,(images,labels) in enumerate(testloader,0):
    images=images.to(device)
    labels=labels.to(device)
    outputs=model(images)
    test_acc += (torch.argmax(outputs,1).flatten()==labels).type(torch.float).mean().item()
    
    preds= torch.argmax(outputs,1).flatten()
    print('labels',labels)
    print('pred',preds)
    
print("Test ACC: %0.2f"%(test_acc/i))


labels tensor([5, 0, 4, 1, 9, 2, 1, 3, 1, 4, 3, 5, 3, 6, 1, 7, 2, 8, 6, 9, 4, 0, 9, 1,
        1, 2, 4, 3, 2, 7, 3, 8], device='cuda:0')
pred tensor([5, 0, 4, 1, 9, 2, 1, 3, 1, 4, 3, 5, 3, 6, 1, 7, 2, 8, 6, 9, 4, 0, 9, 1,
        1, 2, 4, 3, 2, 7, 3, 8], device='cuda:0')
labels tensor([6, 9, 0, 5, 6, 0, 7, 6, 1, 8, 7, 9, 3, 9, 8, 5, 9, 3, 3, 0, 7, 4, 9, 8,
        0, 9, 4, 1, 4, 4, 6, 0], device='cuda:0')
pred tensor([6, 9, 0, 5, 6, 0, 7, 6, 1, 8, 7, 9, 3, 9, 8, 5, 9, 3, 3, 0, 7, 4, 7, 8,
        0, 9, 4, 1, 4, 4, 6, 0], device='cuda:0')
labels tensor([4, 5, 6, 1, 0, 0, 1, 7, 1, 6, 3, 0, 2, 1, 1, 7, 9, 0, 2, 6, 7, 8, 3, 9,
        0, 4, 6, 7, 4, 6, 8, 0], device='cuda:0')
pred tensor([4, 5, 6, 1, 0, 0, 1, 7, 1, 6, 3, 0, 2, 1, 1, 7, 5, 0, 2, 6, 7, 8, 3, 9,
        0, 4, 6, 7, 4, 6, 8, 0], device='cuda:0')
labels tensor([7, 8, 3, 1, 5, 7, 1, 7, 1, 1, 6, 3, 0, 2, 9, 3, 1, 1, 0, 4, 9, 2, 0, 0,
        2, 0, 2, 7, 1, 8, 6, 4], device='cuda:0')
pred tensor([7, 8, 3, 1, 5, 7, 1, 7, 1, 1, 6, 3

        0, 5, 6, 4, 4, 2, 4, 4], device='cuda:0')
pred tensor([5, 1, 7, 8, 5, 0, 3, 4, 7, 7, 5, 7, 8, 6, 9, 3, 8, 6, 1, 0, 9, 7, 1, 3,
        0, 5, 6, 4, 4, 2, 4, 4], device='cuda:0')
labels tensor([3, 1, 7, 7, 6, 0, 3, 6, 0, 7, 1, 1, 4, 9, 4, 3, 4, 8, 2, 2, 1, 8, 7, 0,
        8, 1, 0, 7, 6, 3, 7, 7], device='cuda:0')
pred tensor([3, 1, 7, 7, 6, 0, 3, 6, 0, 7, 1, 1, 4, 9, 4, 3, 4, 8, 2, 2, 1, 8, 7, 0,
        8, 1, 0, 7, 6, 3, 7, 7], device='cuda:0')
labels tensor([5, 8, 8, 9, 0, 0, 4, 1, 5, 2, 2, 3, 9, 4, 9, 5, 0, 6, 7, 7, 1, 8, 0, 2,
        2, 0, 4, 1, 1, 2, 7, 3], device='cuda:0')
pred tensor([5, 8, 8, 9, 0, 0, 4, 1, 5, 2, 2, 3, 9, 4, 9, 5, 0, 6, 7, 7, 1, 8, 0, 7,
        2, 0, 4, 1, 1, 2, 7, 3], device='cuda:0')
labels tensor([9, 7, 2, 8, 1, 9, 5, 8, 8, 1, 9, 8, 3, 1, 6, 5, 7, 4, 2, 7, 0, 3, 0, 4,
        1, 1, 7, 9, 1, 1, 8, 5], device='cuda:0')
pred tensor([9, 7, 2, 8, 1, 9, 5, 8, 8, 1, 9, 8, 3, 1, 6, 5, 7, 4, 2, 6, 0, 3, 0, 4,
        1, 1, 7, 9, 1, 1, 8, 5], device='cuda:0')

        0, 3, 1, 1, 2, 7, 6, 9], device='cuda:0')
pred tensor([6, 7, 0, 3, 6, 5, 3, 2, 4, 1, 0, 4, 2, 4, 1, 3, 4, 0, 6, 0, 5, 5, 4, 7,
        0, 3, 1, 1, 2, 7, 6, 9], device='cuda:0')
labels tensor([5, 6, 0, 3, 5, 3, 1, 0, 1, 5, 4, 6, 9, 6, 3, 8, 5, 7, 6, 3, 1, 5, 9, 0,
        2, 1, 1, 2, 8, 3, 8, 4], device='cuda:0')
pred tensor([5, 6, 0, 3, 5, 3, 1, 0, 1, 5, 4, 6, 9, 6, 3, 8, 5, 7, 6, 3, 1, 5, 9, 0,
        2, 1, 1, 2, 8, 3, 8, 4], device='cuda:0')
labels tensor([0, 5, 5, 6, 5, 7, 7, 8, 0, 9, 6, 0, 9, 1, 8, 2, 3, 3, 4, 4, 3, 5, 5, 6,
        5, 7, 1, 8, 8, 9, 0, 0], device='cuda:0')
pred tensor([0, 5, 5, 6, 5, 7, 7, 8, 0, 9, 6, 0, 9, 1, 8, 2, 3, 3, 4, 4, 3, 5, 5, 6,
        5, 7, 1, 8, 8, 9, 0, 0], device='cuda:0')
labels tensor([2, 1, 6, 2, 7, 3, 7, 4, 7, 7, 7, 8, 1, 9, 4, 8, 6, 5, 1, 7, 0, 9, 1, 5,
        9, 0, 8, 8, 1, 3, 2, 5], device='cuda:0')
pred tensor([2, 1, 6, 2, 7, 3, 7, 4, 7, 7, 7, 8, 1, 9, 4, 8, 6, 5, 1, 7, 0, 9, 1, 5,
        9, 0, 8, 8, 1, 3, 2, 5], device='cuda:0')

        9, 8, 9, 6, 6, 4, 9, 9], device='cuda:0')
pred tensor([6, 5, 4, 8, 0, 2, 7, 1, 3, 7, 1, 9, 4, 1, 8, 1, 9, 6, 6, 2, 5, 1, 4, 3,
        9, 8, 9, 6, 6, 4, 9, 9], device='cuda:0')
labels tensor([7, 4, 8, 2, 3, 4, 4, 4, 7, 6, 7, 2, 5, 5, 2, 5, 8, 0, 3, 2, 0, 8, 0, 0,
        2, 6, 7, 8, 8, 1, 8, 7], device='cuda:0')
pred tensor([7, 4, 8, 2, 3, 4, 4, 4, 7, 6, 7, 2, 5, 5, 7, 5, 8, 0, 3, 2, 0, 8, 0, 0,
        2, 6, 7, 8, 8, 1, 8, 7], device='cuda:0')
labels tensor([3, 1, 3, 6, 3, 7, 0, 6, 1, 8, 4, 7, 7, 4, 3, 9, 2, 3, 6, 3, 1, 5, 6, 8,
        7, 0, 4, 3, 0, 7, 9, 9], device='cuda:0')
pred tensor([3, 1, 3, 6, 3, 7, 0, 6, 1, 8, 4, 7, 7, 4, 3, 9, 2, 3, 6, 3, 1, 5, 6, 8,
        7, 0, 4, 3, 0, 7, 9, 9], device='cuda:0')
labels tensor([4, 7, 3, 7, 1, 9, 9, 1, 7, 6, 0, 7, 5, 8, 6, 0, 9, 1, 2, 2, 9, 3, 0, 4,
        1, 5, 6, 6, 1, 7, 0, 8], device='cuda:0')
pred tensor([4, 7, 3, 7, 1, 9, 9, 1, 7, 6, 0, 7, 5, 8, 6, 0, 9, 1, 2, 2, 9, 3, 0, 4,
        1, 5, 6, 6, 1, 7, 0, 8], device='cuda:0')

        8, 1, 8, 1, 1, 5, 3, 9], device='cuda:0')
pred tensor([6, 6, 7, 0, 4, 3, 6, 6, 8, 4, 0, 8, 6, 4, 9, 5, 1, 0, 0, 3, 7, 9, 6, 2,
        8, 1, 8, 1, 1, 5, 3, 9], device='cuda:0')
labels tensor([7, 4, 3, 2, 7, 0, 0, 4, 8, 4, 3, 0, 2, 9, 3, 6, 0, 9, 7, 2, 5, 2, 8, 1,
        5, 7, 4, 9, 2, 5, 5, 1], device='cuda:0')
pred tensor([7, 4, 3, 2, 7, 0, 0, 4, 8, 4, 3, 0, 2, 9, 3, 6, 0, 9, 7, 2, 5, 2, 8, 1,
        5, 7, 4, 9, 2, 5, 5, 1], device='cuda:0')
labels tensor([3, 1, 5, 9, 8, 6, 9, 0, 8, 1, 4, 8, 6, 4, 4, 6, 6, 8, 0, 8, 4, 2, 8, 6,
        8, 3, 4, 2, 4, 9, 8, 2], device='cuda:0')
pred tensor([3, 1, 5, 9, 8, 6, 9, 0, 8, 1, 4, 8, 6, 4, 9, 6, 6, 8, 0, 8, 4, 2, 8, 6,
        8, 3, 4, 2, 4, 9, 8, 2], device='cuda:0')
labels tensor([8, 4, 7, 0, 9, 9, 1, 0, 7, 3, 3, 3, 6, 5, 5, 8, 0, 4, 1, 7, 5, 6, 9, 2,
        5, 2, 9, 7, 3, 7, 2, 0], device='cuda:0')
pred tensor([8, 4, 7, 0, 9, 9, 1, 0, 7, 3, 3, 3, 6, 5, 5, 8, 0, 4, 1, 7, 5, 6, 9, 2,
        5, 2, 9, 7, 3, 7, 2, 0], device='cuda:0')

...

        5, 7, 3, 1, 9, 0, 7, 1], device='cuda:0')
pred tensor([6, 5, 9, 8, 7, 3, 8, 5, 6, 4, 6, 7, 3, 6, 7, 5, 4, 1, 2, 2, 8, 2, 8, 7,
        5, 7, 3, 1, 9, 0, 7, 1], device='cuda:0')
labels tensor([6, 2, 1, 3, 4, 4, 1, 5, 7, 6, 3, 7, 0, 8, 5, 9, 1, 0, 0, 1, 6, 2, 6, 3,
        3, 4, 6, 5, 3, 6, 7, 7], device='cuda:0')
pred tensor([6, 2, 1, 3, 4, 4, 1, 5, 7, 6, 3, 7, 0, 8, 5, 9, 1, 0, 5, 1, 6, 2, 6, 3,
        3, 4, 6, 5, 3, 6, 7, 7], device='cuda:0')
labels tensor([0, 8, 3, 9, 4, 0, 3, 1, 1, 2, 5, 3, 1, 4, 0, 5, 3, 6, 2, 7, 3, 8, 2, 9,
        0, 3, 8, 2, 7, 9, 0, 3], device='cuda:0')
pred tensor([0, 8, 3, 9, 4, 0, 3, 1, 1, 2, 5, 3, 1, 4, 0, 5, 3, 6, 2, 7, 3, 8, 2, 9,
        0, 3, 8, 2, 7, 9, 0, 3], device='cuda:0')
labels tensor([3, 2, 7, 1, 4, 4, 3, 5, 7, 5, 2, 2, 8, 3, 0, 2, 7, 1, 7, 3, 8, 9, 4, 7,
        7, 2, 0, 1, 7, 1, 0, 8], device='cuda:0')
pred tensor([3, 2, 7, 1, 4, 4, 3, 5, 7, 5, 2, 2, 8, 3, 0, 2, 7, 1, 7, 3, 8, 9, 4, 7,
        7, 2, 0, 1, 7, 1, 0, 8], device='cuda:0')

        7, 7, 5, 9, 3, 2, 8, 0], device='cuda:0')
pred tensor([3, 2, 2, 7, 0, 9, 6, 1, 8, 0, 3, 1, 7, 0, 9, 4, 3, 0, 3, 1, 6, 0, 1, 4,
        7, 7, 5, 9, 3, 2, 8, 0], device='cuda:0')
labels tensor([0, 6, 2, 2, 0, 6, 4, 2, 2, 2, 7, 9, 6, 9, 9, 0, 8, 1, 1, 2, 4, 3, 1, 4,
        6, 5, 6, 6, 8, 7, 8, 8], device='cuda:0')
pred tensor([0, 6, 4, 2, 0, 6, 4, 2, 2, 2, 7, 9, 6, 9, 9, 0, 8, 1, 1, 2, 4, 3, 1, 4,
        6, 5, 6, 6, 8, 7, 8, 8], device='cuda:0')
labels tensor([3, 0, 7, 1, 1, 2, 5, 3, 1, 4, 2, 5, 4, 6, 1, 7, 6, 8, 1, 9, 8, 0, 7, 1,
        1, 2, 8, 3, 8, 4, 2, 5], device='cuda:0')
pred tensor([3, 0, 7, 1, 1, 2, 5, 3, 1, 4, 2, 5, 4, 6, 1, 7, 6, 8, 1, 9, 8, 0, 7, 1,
        1, 2, 8, 3, 8, 4, 2, 5], device='cuda:0')
labels tensor([5, 6, 2, 7, 8, 8, 5, 9, 3, 8, 1, 3, 6, 4, 9, 7, 6, 8, 5, 6, 1, 3, 6, 4,
        4, 0, 3, 9, 3, 7, 7, 1], device='cuda:0')
pred tensor([5, 6, 2, 7, 8, 8, 5, 9, 3, 8, 1, 3, 6, 4, 9, 7, 6, 8, 0, 6, 1, 3, 6, 4,
        4, 0, 3, 9, 3, 7, 7, 1], device='cuda:0')

        5, 4, 3, 8, 4, 8, 5, 7], device='cuda:0')
pred tensor([1, 7, 3, 5, 3, 8, 8, 3, 0, 4, 3, 7, 7, 6, 7, 3, 3, 2, 9, 3, 7, 8, 4, 9,
        5, 4, 3, 8, 4, 8, 5, 7], device='cuda:0')
labels tensor([1, 1, 2, 0, 2, 8, 2, 7, 7, 7, 2, 9, 0, 2, 9, 3, 2, 4, 4, 6, 3, 1, 5, 1,
        5, 4, 4, 5, 3, 5, 3, 0], device='cuda:0')
pred tensor([1, 1, 2, 0, 3, 8, 2, 7, 7, 7, 2, 9, 0, 2, 9, 3, 2, 4, 4, 6, 3, 1, 5, 1,
        5, 4, 4, 5, 3, 5, 3, 0], device='cuda:0')
labels tensor([1, 4, 1, 8, 6, 7, 2, 4, 7, 9, 2, 2, 2, 3, 3, 6, 0, 4, 6, 1, 0, 2, 2, 4,
        0, 1, 0, 5, 0, 4, 4, 2], device='cuda:0')
pred tensor([1, 4, 1, 8, 6, 7, 2, 4, 7, 9, 2, 2, 2, 3, 3, 6, 0, 4, 6, 1, 0, 2, 2, 4,
        0, 1, 0, 5, 0, 4, 4, 2], device='cuda:0')
labels tensor([1, 0, 7, 4, 6, 8, 0, 6, 1, 1, 4, 9, 5, 3, 6, 2, 0, 0, 2, 0, 0, 2, 2, 5,
        2, 6, 8, 9, 0, 3, 8, 6], device='cuda:0')
pred tensor([1, 0, 7, 4, 6, 8, 0, 6, 1, 1, 4, 9, 5, 3, 6, 2, 0, 0, 2, 0, 0, 2, 2, 5,
        2, 6, 8, 9, 0, 3, 8, 6], device='cuda:0')

        2, 2, 5, 8, 1, 5, 7, 4], device='cuda:0')
pred tensor([7, 6, 4, 7, 1, 3, 4, 8, 4, 9, 7, 0, 7, 7, 3, 7, 1, 3, 5, 4, 1, 5, 0, 5,
        2, 2, 5, 8, 1, 5, 7, 4], device='cuda:0')
labels tensor([8, 6, 0, 6, 4, 6, 6, 7, 7, 9, 2, 1, 9, 8, 0, 2, 1, 1, 9, 2, 6, 6, 7, 5,
        1, 3, 4, 4, 4, 7, 9, 9], device='cuda:0')
pred tensor([8, 6, 0, 6, 4, 6, 6, 7, 7, 9, 2, 1, 9, 8, 0, 2, 1, 1, 9, 2, 6, 6, 7, 5,
        1, 3, 4, 4, 4, 7, 9, 9], device='cuda:0')
labels tensor([0, 4, 6, 0, 9, 0, 8, 5, 5, 9, 1, 0, 8, 1, 6, 2, 5, 3, 7, 4, 0, 5, 9, 6,
        1, 7, 3, 8, 5, 9, 1, 0], device='cuda:0')
pred tensor([0, 4, 6, 0, 9, 0, 8, 5, 5, 9, 1, 0, 8, 1, 6, 2, 5, 3, 7, 4, 0, 5, 9, 6,
        1, 7, 3, 8, 5, 9, 1, 0], device='cuda:0')
labels tensor([6, 1, 3, 2, 4, 3, 9, 4, 1, 5, 6, 6, 5, 7, 1, 8, 2, 9, 3, 0, 7, 1, 4, 2,
        9, 3, 2, 4, 1, 5, 0, 6], device='cuda:0')
pred tensor([6, 1, 3, 2, 4, 3, 9, 4, 1, 5, 6, 6, 5, 7, 1, 8, 2, 9, 3, 0, 7, 1, 4, 2,
        9, 3, 2, 4, 1, 5, 0, 6], device='cuda:0')

        0, 2, 1, 3, 1, 7, 2, 1], device='cuda:0')
pred tensor([0, 1, 9, 2, 3, 7, 9, 1, 2, 5, 1, 7, 2, 8, 7, 0, 1, 9, 6, 4, 1, 2, 3, 7,
        0, 2, 1, 3, 1, 7, 2, 1], device='cuda:0')
labels tensor([6, 8, 0, 8, 9, 1, 6, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 7, 8,
        9, 0, 1, 2, 3, 4, 5, 6], device='cuda:0')
pred tensor([6, 8, 0, 8, 9, 1, 6, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 7, 8,
        9, 0, 1, 2, 3, 4, 5, 6], device='cuda:0')
labels tensor([7, 8, 6, 4, 2, 6, 4, 7, 5, 5, 4, 7, 8, 9, 2, 9, 3, 9, 3, 8, 2, 0, 5, 0,
        7, 0, 4, 2, 6, 5, 3, 5], device='cuda:0')
pred tensor([7, 8, 6, 4, 2, 6, 4, 7, 5, 5, 4, 7, 8, 9, 2, 9, 3, 9, 3, 8, 2, 0, 5, 0,
        7, 0, 4, 2, 6, 5, 3, 5], device='cuda:0')
labels tensor([3, 8, 0, 0, 3, 4, 1, 5, 3, 0, 8, 3, 0, 6, 2, 7, 1, 1, 8, 1, 7, 1, 3, 8,
        9, 7, 6, 7, 4, 1, 6, 7], device='cuda:0')
pred tensor([3, 8, 0, 0, 3, 4, 1, 5, 3, 0, 8, 3, 0, 6, 2, 7, 1, 1, 8, 1, 7, 1, 3, 8,
        9, 7, 6, 7, 4, 1, 6, 7], device='cuda:0')